In [2]:
import ultralytics
import os
from dotenv import load_dotenv
from roboflow import Roboflow
import cv2
from ultralytics import YOLO


In [3]:
load_dotenv()
ultralytics.checks()

Ultralytics 8.3.96 🚀 Python-3.13.2 torch-2.6.0+cu124 CPU (AMD Ryzen 9 6900HX with Radeon Graphics)
Setup complete ✅ (16 CPUs, 60.6 GB RAM, 558.9/937.3 GB disk)


# Download dataset with 2096 butterflies.

In [6]:
rf = Roboflow(api_key=os.getenv('ROBOFLOW_API_KEY'))
workspace = rf.workspace("firstwp-gzzwu")
project = workspace.project("butterflies-vs-moths-detection-duc4y")
version = project.version(1)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...
Exporting format yolov11 in progress : 85.0%
Version export complete for yolov11 format



Extracting Dataset Version Zip to Butterflies-vs-Moths-Detection-1 in yolov11:: 100%|██████████| 8555/8555 [00:00<00:00, 11072.89it/s]


# Train it using yolo-11-s and image size of 640

In [7]:
# !yolo task=detect mode=train model=yolo11s.pt data={dataset.location}/data.yaml epochs=10 imgsz=640 plots=True

/usr/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8): No such file or directory
New https://pypi.org/project/ultralytics/8.3.97 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.96 🚀 Python-3.13.2 torch-2.6.0+cu124 CPU (AMD Ryzen 9 6900HX with Radeon Graphics)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/home/vmchura/DataspellProjects/ImageDetection/Butterflies-vs-Moths-Detection-1/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=Fals

In [ ]:
# Evaluate the model

In [8]:
!yolo task=detect mode=val model=runs/zipped_train_two_datasets/weights/best.pt data={dataset.location}/data.yaml

/usr/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8): No such file or directory
Ultralytics 8.3.96 🚀 Python-3.13.2 torch-2.6.0+cu124 CPU (AMD Ryzen 9 6900HX with Radeon Graphics)
YOLO11s summary (fused): 100 layers, 9,413,961 parameters, 0 gradients, 21.3 GFLOPs
val: Scanning /home/vmchura/DataspellProjects/ImageDetection/Butterflies-vs-Moth
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all        809        810      0.846      0.876      0.909      0.743
                     1        650        650      0.969       0.92      0.984      0.832
                     2        159        160      0.723      0.831      0.834      0.654
Speed: 0.8ms preprocess, 83.6ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to /home/vmchura/DataspellProjects/ImageDetection/runs/detect/val2
💡 Learn more at https://docs.ultralytics.com/modes/val


In [10]:
!yolo task=detect mode=predict model=runs/zipped_train_two_datasets/weights/best.pt conf=0.6 source=report_images_test save=True

/usr/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8): No such file or directory
Ultralytics 8.3.96 🚀 Python-3.13.2 torch-2.6.0+cu124 CPU (AMD Ryzen 9 6900HX with Radeon Graphics)
YOLO11s summary (fused): 100 layers, 9,413,961 parameters, 0 gradients, 21.3 GFLOPs

image 1/6 /home/vmchura/DataspellProjects/ImageDetection/report_images_test/negative_imagen_723.jpg: 640x640 1 2, 78.9ms
image 2/6 /home/vmchura/DataspellProjects/ImageDetection/report_images_test/negative_imagen_880.jpg: 640x608 1 1, 1 2, 79.2ms
image 3/6 /home/vmchura/DataspellProjects/ImageDetection/report_images_test/photogram_16.jpeg: 384x640 1 1, 1 2, 50.9ms
image 4/6 /home/vmchura/DataspellProjects/ImageDetection/report_images_test/photogram_93.jpeg: 384x640 1 1, 42.6ms
image 5/6 /home/vmchura/DataspellProjects/ImageDetection/report_images_test/positive_imagen_45.jpeg: 640x640 1 1, 70.4ms
image 6/6 /home/vmchura/DataspellProjects/ImageDetection/report_images_test/positive_imagen_888.jpeg: 640x64

In [12]:
model_trained = YOLO("runs/zipped_train_two_datasets/weights/best.pt")

In [13]:
actual_results = []
expected_predictions = []

For the challenge, to evaluate the metrics for the classification problem I used accuracy and f1 score.

In [14]:
for expected_label, directory in [(0, os.getenv('NEGATIVE_IMAGES_CHALLENGE_PATH')), (1, os.getenv('POSITIVE_IMAGES_CHALLENGE_PATH'))]:
    list_images = os.listdir(directory)
    expected_predictions.extend([expected_label] * len(list_images))
    for image_name in list_images:
        results = model_trained(os.path.join(directory, image_name), conf=0.6)
        r = [1 for k in results if len(k.boxes.cls.cpu().numpy()) > 0]
        actual_results.append(1 if len(r) > 0 else 0)


image 1/1 /home/vmchura/DataspellProjects/nuwe-data-dl2-8134/data/train/0/imagen_549.jpg: 576x640 1 1, 89.1ms
Speed: 2.4ms preprocess, 89.1ms inference, 0.6ms postprocess per image at shape (1, 3, 576, 640)

image 1/1 /home/vmchura/DataspellProjects/nuwe-data-dl2-8134/data/train/0/imagen_1360.jpg: 640x608 1 1, 87.5ms
Speed: 1.9ms preprocess, 87.5ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 608)

image 1/1 /home/vmchura/DataspellProjects/nuwe-data-dl2-8134/data/train/0/imagen_228.jpg: 608x640 1 1, 90.9ms
Speed: 1.8ms preprocess, 90.9ms inference, 0.5ms postprocess per image at shape (1, 3, 608, 640)

image 1/1 /home/vmchura/DataspellProjects/nuwe-data-dl2-8134/data/train/0/imagen_1268.jpg: 576x640 1 1, 60.9ms
Speed: 1.7ms preprocess, 60.9ms inference, 0.4ms postprocess per image at shape (1, 3, 576, 640)

image 1/1 /home/vmchura/DataspellProjects/nuwe-data-dl2-8134/data/train/0/imagen_1435.jpg: 640x640 1 2, 99.3ms
Speed: 2.0ms preprocess, 99.3ms inference, 0.5ms postp

In [15]:
from sklearn.metrics import accuracy_score, f1_score

In [16]:
accuracy = accuracy_score(expected_predictions, actual_results)
f1 = f1_score(expected_predictions, actual_results, zero_division=0)
print(accuracy)
print(f1)

0.520952380952381
0.6189393939393939
